In [2]:
from encode_data import read_from_file
from encode_data import generate_slices
from encode_data import generate_fourier_descriptions
from feature_generation.definitions import Atom, Point
from feature_generation.contour_descriptor import fourier_descriptor

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from feature_generation.alignment import align_catalyst
import matplotlib.pyplot as plt
from pyefd import reconstruct_contour, plot_efd, normalize_efd, calculate_dc_coefficients

import numpy as np
from scipy.spatial.transform import Rotation as R
import copy

# Test rotational invariance

In [3]:
molecule = read_from_file("test_data/ir_tbp_1_dft-asme3_1_dft-asme3_1_dft-co_1_bromide_1_smi1_1_s_1_ts.xyz")
#print(molecule)

rot3d = R.from_quat([0, 2, np.sin(np.pi/4), np.cos(np.pi/4)]).as_matrix()
molecule_rot = [Atom(ele, rot3d @ loc, rad) for (ele, loc, rad) in molecule]
#print(molecule_rot)

In [4]:
slices = generate_slices(molecule, 0.5, -5, 5, 0.01)
print(slices[9])

slices_rot = generate_slices(molecule_rot, 0.5, -5, 5, 0.01)
print(slices_rot[9])

[[3.03586692 2.42738231]
 [3.03579067 2.44263237]
 [3.03556193 2.45788089]
 ...
 [3.03553737 2.39568004]
 [3.03577816 2.41092838]
 [3.03586645 2.42617837]]
[[-0.2591793  -2.04771334]
 [-0.25925294 -2.03298637]
 [-0.25947384 -2.01826087]
 ...
 [-0.25950283 -2.07858106]
 [-0.25926777 -2.06385578]
 [-0.25917998 -2.04912889]]


In [5]:
def q(x):
    fig = plt.figure()
    ax1 = fig.add_subplot(1,2,1)
    ax1.scatter(*zip(*list(slices[x])))
    ax1.set_aspect('equal')

    ax2 = fig.add_subplot(1,2,2)
    ax2.scatter(*zip(*list(slices_rot[x])))
    ax2.set_aspect('equal')
    plt.show()

interact(q, x=widgets.IntSlider(min=1, max=len(slices), step=1, value=9));

interactive(children=(IntSlider(value=9, description='x', max=20, min=1), Output()), _dom_classes=('widget-int…

In [6]:
fourier = generate_fourier_descriptions(slices, 10)
print(fourier)

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.0000

In [13]:
def q(x):
    fig = plt.figure()
    ax1 = fig.add_subplot(1,2,1)
    ax1.scatter(*zip(*list(slices[x])))
    ax1.set_aspect('equal')
    
    reshaped = fourier[x].reshape(-1,4)
    A0 = reshaped[0][1]
    C0 = reshaped[0][2]

    reshaped[0][1] = 0
    reshaped[0][2] = 0

    contour = reconstruct_contour(reshaped, (A0, C0))
    ax2 = fig.add_subplot(1,2,2)
    ax2.scatter(*zip(*list(contour)))
    ax2.set_aspect('equal')
    plt.show()
    
    plot_efd(fourier[x].reshape(-1,4), (A0, C0))

interact(q, x=widgets.IntSlider(min=1, max=len(fourier-1), step=1, value=9));

interactive(children=(IntSlider(value=9, description='x', max=20, min=1), Output()), _dom_classes=('widget-int…